In [ ]:
import pandas as pd
import numpy as np
import json
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import PIL
import skimage.io
import seaborn as sn
from collections import Counter
import tensorflow as tf
import collections
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

This part is inspired from https://www.kaggle.com/mpwolke/kaggle-herbarium-2022-long-tailed-learning

In [ ]:

TRAIN_DIR = "../input/herbarium-2022-fgvc9/train_images/"
TEST_DIR = "../input/herbarium-2022-fgvc9/test_images/"

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

# Exploration

In [ ]:
train_meta.keys()

In [ ]:
print('the number of images in the training set : ', len(train_meta['images']))
print('the number of annotations in the training set :', len(train_meta['annotations']))

In [ ]:
print('The total number of classes : ', len(set([annotation["category_id"] for annotation in train_meta["annotations"]])))

In [ ]:
ids = []
categories = []
paths = []

for annotation, image in zip(train_meta["annotations"], train_meta["images"]):
    #assert annotation["image_id"] == image["id"]
    ids.append(image["image_id"]) #Read above print metadata samples from each key
    categories.append(annotation["category_id"])
    paths.append(image["file_name"])
        
df_meta = pd.DataFrame({"id": ids, "category": categories, "path": paths})

In [ ]:
df_meta

In [ ]:
df_meta['category'].value_counts()

unbalanced classes !

In [ ]:
d_categories = {category["category_id"]: category["scientificName"] for category in train_meta["categories"]}
d_families = {category["category_id"]: category["family"] for category in train_meta["categories"]}
d_genus = {category["category_id"]: category["genus"] for category in train_meta["categories"]}
d_species = {category["category_id"]: category["species"] for category in train_meta["categories"]}

df_meta["category_name"] = df_meta["category"].map(d_categories)
df_meta["family_name"] = df_meta["category"].map(d_families)
df_meta["genus_name"] = df_meta["category"].map(d_genus)
df_meta["species_name"] = df_meta["category"].map(d_species)
df_meta

In [ ]:
def visualize_train_batch(paths, categories, families, genus, species):
    plt.figure(figsize=(16, 16))
    
    for ind, info in enumerate(zip(paths, categories, families, genus, species)):
        path, category, family, genus, species = info
        
        plt.subplot(2, 3, ind + 1)
        
        image = cv2.imread(os.path.join("../input/herbarium-2022-fgvc9/train_images", path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        
        plt.title(
            f"FAMILY: {family} GENUS: {genus}\n{species}", 
            fontsize=10,
        )
        plt.axis("off")
    
    plt.show()

In [ ]:
def visualize_by_id(df, _id=None):
    tmp = df.sample(6)
    if _id is not None:
        tmp = df[df["category"] == _id].sample(6)

    visualize_train_batch(
        tmp["path"].tolist(), 
        tmp["category_name"].tolist(),
        tmp["family_name"].tolist(),
        tmp["genus_name"].tolist(),
        tmp["species_name"].tolist(),
    )

In [ ]:
for i in range(4):
    visualize_by_id(df_meta, i)
    

In [ ]:
df_meta

# Preprocessing

In [ ]:
img = os.path.join("../input/herbarium-2022-fgvc9/train_images", '000/00/00000__001.jpg')

In [ ]:
cv2.imread(img)

In [ ]:
cv2.imread(img).shape

In [ ]:
img = cv2.imread(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
w =299
h =299
res_img = cv2.resize(img, (w,h), interpolation = cv2.INTER_AREA)

In [ ]:
res_img.shape

In [ ]:
fig = plt.figure(figsize=(16,6))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("original size : {}".format(img.shape))
plt.subplot(1, 2, 2)

plt.imshow(res_img)
plt.title("Resized shape : {}".format(res_img.shape))
plt.suptitle("Changing the image size", fontsize=20)
plt.show()

In [ ]:
img_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
plt.figure(figsize = (16,6))
plt.subplot(1, 2, 1)

plt.hist(img_yuv.flatten(), bins=range(256))
plt.title('Luminance distribution')
plt.subplot(1, 2, 2)
plt.imshow(img_yuv)

plt.show()

In [ ]:
# Equalization
img_yuv[:, : ,0] = cv2.equalizeHist(img_yuv[:, : ,0])
img_equ = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

In [ ]:
fig = plt.figure(figsize=(16,6))
plt.subplot(1, 2,1)
plt.hist(img_equ.flatten(), bins=range(256))
plt.title('Luminance distribution after equalization')
plt.subplot(1, 2, 2)

plt.imshow(img_equ[:, : , ::-1])
plt.title('Image after equalization')
plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
     zoom_range=0.2,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest', cval = 125)

x = img
x = x.reshape((1,) + x.shape)

aug_ = datagen.flow(x)

In [ ]:
aug_images = [next(aug_)[0].astype(np.uint8) for i in range(12)]

fig, axes = plt.subplots(3,4,figsize= (20,20))
axes = axes.flatten()
for ig, ax in zip(aug_images,axes):
    ax.imshow(ig)
    ax.axis('off')
    
plt.suptitle("Data augmentation",fontsize=24)
plt.show()

In [ ]:
def preprocess_cnn(categories, num_categories, w,h):
    
    """"
    Preprocessing images for neural networks
    """
    """
    params : categories and num_categories for fils to be processed
    w =  width of the image
    h = height of the image
    
    """
    
    list_img = []
    labels = []
    for cat,num_cat in zip(categories, num_categories):
        for ig in os.listdir(os.path.join("../input/herbarium-2022-fgvc9/train_images", cat, num_cat)) :
            img = cv2.imread(os.path.join("../input/herbarium-2022-fgvc9/train_images", cat, num_cat, ig)) # read image
            img = cv2.resize(img, (w,h), interpolation=cv2.INTER_LINEAR) # resize image
            #equilization
            img_yuv = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
            img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
            img_equ = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

            list_img.append(img_equ)
            labels.append(ig.split('__')[0])
        
    return list_img, labels
              

In [ ]:
df_meta['cat_first'] = df_meta['path'].apply(lambda x : x.split('/')[0])
df_meta['cat_second'] = df_meta['path'].apply(lambda x : x.split('/')[1])

In [ ]:
df_meta.head()

We will train the model on only the 20 most known classes.

In [ ]:
index_most = df_meta['category'].value_counts().head(20).index.tolist()

In [ ]:
df_meta['category_name'].value_counts().head(20).index

In [ ]:
df_meta_most = df_meta[df_meta['category'].isin(index_most)]

In [ ]:
df_meta_most['category'].unique()

In [ ]:
df_meta_most['cat_first'].unique()

In [ ]:
cat_first_ = ['002', '009', '011', '011', '027', '028', '028', '040', '046', '046', '087', '088','088', '100', '108', '109', '125', '125', '125', '125']

In [ ]:
len(cat_first_)

In [ ]:
df_meta_most['cat_second'].unique()

In [ ]:
df_meta_most['cat_second'].unique().shape

# Model

In [ ]:
X, y = preprocess_cnn(cat_first_, df_meta_most['cat_second'].unique(), 299,299)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
print('the number of processed images :', len(y))

In [ ]:
y[:5]

In [ ]:
#we mix categories

l = np.arange(len(y))
np.random.seed(42)
np.random.shuffle(l)

X = X[l]
y = y[l]

In [ ]:
y[:5]

In [ ]:
X = X.astype(int)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify= y,random_state = 42)

In [ ]:
print('Size of training set', X_train.shape[0])
print('Size of test set', X_test.shape[0])

In [ ]:
from tensorflow import keras
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon())) 

In [ ]:
model = Sequential()

#1er block
model.add(Conv2D(filters=16,kernel_size=(3,3), padding='same',use_bias=False,input_shape=(299,299,3),
         activation = 'relu'))
model.add(Conv2D(filters=16,kernel_size=(3,3), padding='same',use_bias=False,input_shape=(299,299,3),
         activation = 'relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4),strides=(4, 4),padding='same'))
model.add(Dropout(0.25))

# 2eme block
model.add(Conv2D(filters=32,kernel_size=(3,3), padding='same',use_bias=False, activation ='relu'))
model.add(Conv2D(filters=32,kernel_size=(3,3), padding='same',use_bias=False, activation ='relu'))
model.add(BatchNormalization(scale=False))
model.add(MaxPooling2D(pool_size=(4, 4),strides=(4, 4),padding='same'))
model.add(Dropout(0.25))

# 3eme block
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', use_bias=False, activation ='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', use_bias=False, activation ='relu'))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Flatten())


model.add(Dense(128, activation='relu'))
model.add(Dense(len(cat_first_), activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', 
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy', f1_m])

In [ ]:
#train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)


#validation and test
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
history_cnn = model.fit(train_datagen.flow(X_train, y_train,batch_size=16,subset='training'),
    validation_data= train_datagen.flow(X_train, y_train,batch_size=8,subset='validation'),
    batch_size=32, epochs=20,verbose=2)

In [ ]:
plt.figure(figsize =(13,5))
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'], color="orange")
plt.title('CNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()